In [373]:
import requests
from selenium import webdriver
from lxml import etree, html
from itertools import chain
import pandas as pd
import json
import re
from urllib.parse import urlparse

# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>')
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(columns=["url","label"])
browser = webdriver.Chrome("./chromedriver")

url = "http://www.nemoapps.com/phrasebooks/hebrew"
url = "https://freemusicarchive.org/genre/International"
url = "https://sound-effects.bbcrewind.co.uk/search?cat=Nature&resultSize=20"

browser.get(url)
doc = browser.execute_script("return document.body.innerHTML")

print(doc)
parsed_uri = urlparse( url )
base = f'{parsed_uri.scheme}://{parsed_uri.netloc}'

htmlparser = etree.HTMLParser()
tree = etree.fromstring(doc, htmlparser)



WebDriverException: Message: unknown error: Chrome failed to start: crashed.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x55930f8a9b33 <unknown>
#1 0x55930f3726d8 <unknown>
#2 0x55930f39584c <unknown>
#3 0x55930f390fca <unknown>
#4 0x55930f3cbe0a <unknown>
#5 0x55930f3c5f53 <unknown>
#6 0x55930f39ba0a <unknown>
#7 0x55930f39cad5 <unknown>
#8 0x55930f8db2fd <unknown>
#9 0x55930f8f44bb <unknown>
#10 0x55930f8dd0d5 <unknown>
#11 0x55930f8f5145 <unknown>
#12 0x55930f8d0aaf <unknown>
#13 0x55930f911ba8 <unknown>
#14 0x55930f911d28 <unknown>
#15 0x55930f92c48d <unknown>
#16 0x7f794af25609 <unknown>


In [ ]:
def stringify_children(node):
    parts = (list(chain(*(etree.tostring(c, encoding="utf-8").decode().split() for c in node.getchildren()))))
    return ' '.join(filter(None, parts))
    
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext.strip()

def _walk_parents(tag):
    if tag.tag in ["source", "audio"]:
        tag = tag.getparent()
        return _walk_parents(tag)
    else:
        return(tag)

def _detect_tag(link: str, tag: str, attr: str):
    A = tree.xpath(f"//{tag}[@{attr}='{link}']")
    if len(A) > 0:
        return A[0]
    else:
        return None

In [ ]:
string_document = html.fromstring(doc)
links = list(set(string_document.iterlinks()))
audio = []

print(links)

for link in links:
    if link[2].endswith(".mp3"):
        ln = link[2]
        audio.append(ln)

print(len(audio))
#no links found with iterlinks
if len(audio) == 0:
    links = string_document.xpath("//div")
    for link in links:
        
        if link.get("data-track-info") is not None:
            label = json.loads(str(link.get("data-track-info")))
            ln = label["fileUrl"]
        df = df.append({'url':ln, 'label':label}, ignore_index = True)     

else:
    #first tag containing audio link
    L = None
    elem = None
    atrib = None
    #tags to try
    tags = [("source", "src"),("audio", "src"),("div", "data-track-info"),("div", "src")]
    for tg in tags:
        A = _detect_tag(audio[0], tg[0], tg[1])
        if A is not None:
            L = _walk_parents(A)
            elem = tg[0]
            atrib = tg[1]
            break
        else:
            print (f"Exception: cannot extract {tg[0]} tag")

    print(L.tag)
    if L.tag == "td":
        L = L.getparent() # the looping element must be the tr
    #get looping siblings
    loops = [L]
    loops += list(L.itersiblings())
    loops += list(L.itersiblings(preceding=True))
    print(len(loops))

    #iterate loops. extract audio url, text and attributes

    for loop in loops:
        items = list(loop.iter())
        ln = None
        for item in items:
            if item.get(atrib) is not None:
                if item.get(atrib).endswith(".mp3"):
                    ln = item.get(atrib)
                    if not ln.startswith(base):
                        if ln.startswith("/"):
                            ln = base + ln
                        else:
                            ln = url + "/" + ln
                break
        label = cleanhtml(stringify_children(loop))
        df = df.append({'url':ln, 'label':label}, ignore_index = True)

[(<Element style at 0x7f9363b7b860>, None, 'ReithSans_W_Bd.woff2', 194), (<Element style at 0x7f9363b7b860>, None, 'ReithSans_W_Rg.woff', 83), (<Element link at 0x7f9363b7b950>, 'href', '/favicon.ico', 0), (<Element script at 0x7f9363b7b4f0>, 'src', 'https://static.bbci.co.uk/frameworks/requirejs/0.13.0/sharedmodules/require.js', 0), (<Element style at 0x7f9363b7b860>, None, 'ReithSans_W_Bd.woff', 236), (<Element script at 0x7f9363b7b310>, 'src', '/static/js/main.572a7de3.chunk.js', 0), (<Element link at 0x7f94336a3a90>, 'href', '/manifest.json', 0), (<Element img at 0x7f9363b7b9a0>, 'src', 'https://ichef.bbci.co.uk/images/ic/480xn/p08zprnd.jpg', 0), (<Element link at 0x7f9363b7b590>, 'href', '/static/css/2.33d33083.chunk.css', 0), (<Element style at 0x7f9363b7b860>, None, 'ReithSans_W_Rg.woff2', 41), (<Element img at 0x7f9363b7b090>, 'src', '', 0), (<Element link at 0x7f9363b7b810>, 'href', '/static/css/main.e13e286b.chunk.css', 0), (<Element script at 0x7f9363b7b130>, 'src', '/static

In [ ]:


print(df.shape)

df.to_parquet("demo_audio.parquet")

print(df.head())

(4, 2)
                                                                                                                   url  \
0  https://files.freemusicarchive.org/storage-freemusicarchive-org/tracks/YZErKmrB6HYf42epNurYQcS1Qq3eIoMoHfObvXxE.mp3   
1  https://files.freemusicarchive.org/storage-freemusicarchive-org/tracks/YZErKmrB6HYf42epNurYQcS1Qq3eIoMoHfObvXxE.mp3   
2  https://files.freemusicarchive.org/storage-freemusicarchive-org/tracks/YZErKmrB6HYf42epNurYQcS1Qq3eIoMoHfObvXxE.mp3   
3  https://files.freemusicarchive.org/storage-freemusicarchive-org/tracks/YZErKmrB6HYf42epNurYQcS1Qq3eIoMoHfObvXxE.mp3   

                                                                                                                                                                                                                                                                                                                                                                                              